<a href="https://colab.research.google.com/github/barmoshe/InsideTradingProject/blob/master/BarAndOfek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LINKS TO ARTICLES 

1.  Buy Stock With Insiders: How To Track Insider Buying :  https://www.investopedia.com/articles/investing/040915/buy-stock-insiders-how-track-insider-buying.asp#:~:text=The%20SEC's%20Edgar%20database%20allows,use%20databases%20of%20insider%20buying.
2.  Officers, Directors and 10% Shareholders : https://www.sec.gov/education/smallbusiness/goingpublic/officersanddirectors
3.  The Insider Trading Anomaly: Recent Academic Studies : https://www.insidermonkey.com/blog/the-insider-trading-anomaly-recent-academic-studies-591/
4.  Learn from Investing Legends for 2020 & Beyond : https://www.fidelity.com/bin-public/060_www_fidelity_com/documents/learning-center/Presentation_Investing%20legends.pdf

install Yahoo finance API 

In [ ]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


imports to our program:

In [ ]:
import pandas as pd
import sklearn as sk
import yfinance as yf
from datetime import datetime, timedelta



upload file from your computer(dont use !!) : 


In [ ]:
 #from google.colab import files
#uploaded = files.upload()

added my google drive



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


add cvs from my drive


In [39]:
insiders_data_small = pd.read_csv('/content/drive/MyDrive/colab/small_sample.csv')
insiders_data = pd.read_csv('/content/drive/MyDrive/colab/all_insiders_after_split_fix.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)



A	Amended filing

D	Derivative transaction in filing (usually option exercise)

E	Error detected in filing

M	Multiple transactions in filing; earliest reported transaction date and weighted average transaction price


Show preview of the dataframe : 

In [67]:
insiders_data

,Filing Date,Ticker,Insider Name,Title,Trade Type,Price,Qty,Owned,ΔOwn,Value,...,M,D,Year,Month,Day,Weekday,DiffToStart,sharpRatio,Y,aggression
0,17/06/2021 12:13,A,Ancher-Jensen Henrik,6,0,143.66,-43731.0,76902.0,-0.36,-6282527.0,...,0,1,2021,6,17,3,7882,-0.173562,0.001685,0.0
1,26/03/2009 13:10,A,Aspect Capital Ltd,2,0,91.11588,-10975.698,58941.078,-0.16,-1000000.0,...,0,0,2009,3,26,3,3416,-5.105017,-0.009347,0.0
2,14/04/2009 10:32,A,Aspect Capital Ltd,2,0,62.421,-11459.406,47481.672,-0.19,-715307.5823,...,0,0,2009,4,14,1,3435,0.441024,-0.012411,0.0
3,16/06/2004 13:06,A,Barnholt Edward W,10,0,18.648069,-121283.49,78283.806,-0.61,-2261703.0,...,0,1,2004,6,16,2,1672,0.057536,-0.001437,0.0
4,25/08/2005 16:50,A,Barnholt Edward W,4,0,21.459227,-72770.094,79265.202,-0.48,-1561590.0,...,0,1,2005,8,25,3,2107,0.492139,0.001437,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637693,28/04/2020 18:00,ZYXI,Sandgaard Thomas,10,0,15.77,-75143,13982000,-0.01,-1184990,...,1,0,2020,4,28,1,5907,2.49892795,NaN,NaN
637694,01/05/2020 18:32,ZYXI,Sandgaard Thomas,10,0,17.1,-50000,13932000,0,-855055,...,0,0,2020,5,1,4,5910,3.05436918,NaN,NaN
637695,16/07/2020 18:33,ZYXI,Sandgaard Thomas,10,0,22,-1250000,12682000,-0.09,-27500000,...,0,0,2020,7,16,3,5986,-1.021126667,NaN,NaN
637696,10/08/2021 18:39,ZYXI,Sandgaard Thomas,10,0,15.17,-177320,13617420,-0.01,-2689820,...,0,0,2021,8,10,1,6376,-0.600463066,NaN,NaN


 Dataset is now stored in a Pandas Dataframe
 

example to get all rows with specified value : 
same as 
```
SELECT *
FROM insiders_data
WHERE 'Ticker' = 'ABMD'
```

In [ ]:
insiders_data.loc[insiders_data['Ticker'] =='A']

simple example to use Yahoo :

In [ ]:
ticker = yf.Ticker('BILL')
ticker.info

example of actions func returns the Dividends

In [ ]:
ticker.actions

,Dividends,Stock Splits
Date,,


In [78]:
def get_next_month_obj(row):
  current = datetime(row['Year'],row['Month'],row['Day'])
  next_30days=current+timedelta(days=30)
  return next_30days
def create_datetime_obj(row):
   return(datetime(row['Year'],row['Month'],row['Day']))



get te relative max and min 

In [133]:
def get_relative_max(row):#TICKER=A--> row['Ticker']=='A'
  current_ticker = yf.Ticker(row['Ticker'])
  historical = current_ticker.history(start=create_datetime_obj(row)-timedelta(days=5),end=get_next_month_obj(row), interval="1d")
  max=historical['High'].max();
  x=(max-row["Price"])/row['Price']
  print("price="+str(row["Price"]))
  print("max="+str(max))
  #print(x)
  return x
def get_relative_min(row):
  current_ticker = yf.Ticker(row['Ticker'])
  historical = current_ticker.history(start=create_datetime_obj(row)-timedelta(days=5) ,end=get_next_month_obj(row), interval="1d")
  min=historical['Low'].min();
  x=(row["Price"])-min/row['Price']
  print("min="+str(min))

  #print(x)
  return x

In [ ]:
print(insiders_data.iloc[0])

In [ ]:
current_ticker = yf.Ticker('A')
historical = current_ticker.history(start=create_datetime_obj(insiders_data.iloc[0]) ,end=get_next_month_obj(insiders_data.iloc[0]), interval="1d")
historical

here will be a function that gets row and return the data we need about that row from YAHOO to determine the Y .

In [105]:
def get_row_Y(row):
  y=1.0#calculate y here
  relative_max=get_relative_max(row)
  relative_min=get_relative_min(row)
  y=relative_max/relative_min
  #print(row)#for debug
  return y
  

aggression calculate:

In [101]:
def get_aggression(row): 
  aggresion =0
  return aggresion

Itaration on the rows to detemine the Y value

In [137]:
insiders_data_small['Y']=pd.to_numeric(insiders_data_small['Y'])
for index, row in insiders_data_small.iterrows():
  print("------"+str(index)+"------")
  if insiders_data_small.at[index,'D'] ==0:
    insiders_data_small.at[index,'Y'] = float(get_row_Y(row))
    print(insiders_data_small.at[index,'Y'])
  else:
    insiders_data_small.at[index,'Y']=1000000
  print("------"+"------")
    #insiders_data_small.at[index,'aggression']=float(get_aggression(row))

------0------
price=120.99
max=146.85000610351562
min=106.01000213623047
0.0017794516681751255
------------
------1------
price=120.99
max=146.85000610351562
min=106.01000213623047
0.0017794516681751255
------------
------2------
price=120.99
max=146.85000610351562
min=106.01000213623047
0.0017794516681751255
------------
------3------
price=121.0
max=146.85000610351562
min=106.01000213623047
0.0017784674160295152
------------
------4------
------------
------5------
price=6.17
max=6.420000076293945
min=4.809999942779541
0.007516787261870698
------------
------6------
price=98.01
max=102.19000244140625
min=64.80999755859375
0.00043810257636934754
------------
------7------
price=98.01
max=102.19000244140625
min=64.80999755859375
0.00043810257636934754
------------
------8------
price=1.19
max=1.8700000047683716
min=1.100000023841858
2.1512181463378974
------------
------9------
price=98.01
max=102.19000244140625
min=64.80999755859375
0.00043810257636934754
------------
------10------
-

In [139]:
insiders_data_small.iloc[4]

Filing Date                   17/02/2022 20:18
Ticker                                    PUBM
Insider Name             Van Der Zweep Michael
Title                                        4
Trade Type                                   0
Price                                    30.02
Qty                                        -30
Owned                                      124
ΔOwn                                     -0.19
Value                                     -901
InsiderID                              1844595
DiffDate                                     1
2w_MFI                               79.195964
mcclellan_osillator                      -5.99
mcclellan_osillator_s                  -594.41
Open                                 29.540001
Close                                28.219999
High                                 29.799999
Low                                      27.99
Volume                                  721500
PriceCompared                         0.007383
AmountTrades 

**Tasks to do :**

1.   determine the Y possible outcomes it could be sucsessful or unsuccessful transaction or it could be more complex then "black and white"
in short : how do we measure sucsess of a transaction.
2.   detrmine what data do we need to get from YAHOO 
3. to be continued...



alldata code (do not usefor now)


In [ ]:
insiders_data['Y']=pd.to_numeric(insiders_data['Y'])
for index, row in insiders_data.iterrows():
    insiders_data.at[index,'Y'] = float(get_row_Y(row))
    insiders_data.at[index,'aggression']=float(get_aggression(row))

In [150]:
insiders_data_small.iloc[8]


Filing Date              17/02/2022 19:21
Ticker                               IMRA
Insider Name                 Mott David M
Title                                   7
Trade Type                              1
Price                                1.19
Qty                                 45926
Owned                               77752
ΔOwn                                 1.44
Value                               54673
InsiderID                         1219871
DiffDate                                2
2w_MFI                          46.922022
mcclellan_osillator                 -5.99
mcclellan_osillator_s             -594.41
Open                                  1.2
Close                                1.23
High                                 1.24
Low                                  1.15
Volume                             588200
PriceCompared                    -0.03252
AmountTrades                            1
A                                       0
E                                 

In [151]:
current_ticker = yf.Ticker('IMRA')
  #historical = current_ticker.history(start=create_datetime_obj(row) ,end=get_next_month_obj(row), interval="1d")
 # max=historical['High'].max();
 # x=(max-row["Price"])/row['Price']

In [152]:
history=current_ticker.history(start=create_datetime_obj(insiders_data_small.iloc[8])-timedelta(days=5) ,end=get_next_month_obj(insiders_data_small.iloc[8]), interval="1d")

In [156]:
x1=(history['High'].max()-insiders_data_small.iloc[8].Price)/insiders_data_small.iloc[8].Price
print("x1=("+str(history['High'].max())+"-"+str(insiders_data_small.iloc[8].Price)+")/"+str(insiders_data_small.iloc[8].Price)+"="+str(x1))

x1=(1.8700000047683716-1.19)/1.19=0.5714285754356064


In [160]:
x2=(insiders_data_small.iloc[8].Price-history['Low'].min())/insiders_data_small.iloc[8].Price
print("x2=("+str(history['Low'].min())+"-"+str(insiders_data_small.iloc[8].Price)+")/"+str(insiders_data_small.iloc[8].Price)+"="+str(x2))

x2=(1.100000023841858-1.19)/1.19=0.07563023206566558


In [161]:
x1/x2

7.555557610076701